## Set up

In [ ]:
import dask.dataframe as dd
import pandas as pd
import os
import re
import logging
from s3fs import S3FileSystem
from dask.delayed import delayed
from distributed import Client, get_worker
from functools import partial

In [ ]:
key = ''
secret = ''

In [ ]:
s3 = S3FileSystem(key=key, secret=secret)

In [ ]:
keys = s3.walk('data.mindhypertrophy.com/Lending Club')

In [ ]:
client = Client('localhost:8786')

In [ ]:
def read_csv_async(key):
    with s3.open(key, 'rb') as fr:
        df = pd.read_csv(fr, index_col=False)
    worker = get_worker()
    df['worker'] = worker.address
    return df

## Compute

In [ ]:
# create a list of 'future' data frame objects
future_frames = [delayed(read_csv_async)(key) for key in keys]
# create a 'future' DataFrame object
df = dd.from_delayed(future_frames)
# read the data frame in the background
df = client.persist(df)

In [ ]:
df_by_worker = df[['worker', 'loan_amnt']].groupby('worker').sum()

In [ ]:
df_by_grade = df[['grade', 'loan_amnt']].groupby('grade').sum()

In [ ]:
df_by_grade.compute()

In [ ]:
df_by_worker.compute()